In [5]:
import pandas as pd
import psycopg2
import sys
import xarray as xr

from getpass import getpass
from subprocess import call, check_output

List of files to process

In [6]:
path = "/data-uerra/mescan-surfex/temperature/"

In [7]:
len(path)

38

In [8]:
!ls {path}

2t.tiff
consecutive_frost_days_index_per_time_period.tiff
consecutive_summer_days_index_per_time_period.tiff
frost_days_index_per_time_period.tiff
heating_degree_days_per_time_period.tiff
ice_days_index_per_time_period.tiff
mescan-surfex-finland-temperature-2000.nc
mescan-surfex-finland-temperature-2001.nc
mescan-surfex-finland-temperature-2002.nc
mescan-surfex-finland-temperature-2003.nc
mescan-surfex-finland-temperature-2005.nc
mescan-surfex-finland-temperature-2006.nc
mescan-surfex-finland-temperature-2007.nc
mescan-surfex-finland-temperature-2008.nc
mescan-surfex-finland-temperature-2009.nc
mescan-surfex-finland-temperature-2010.nc
mescan-surfex-finland-temperature-2011.nc
mescan-surfex-finland-temperature-2012.nc
mescan-surfex-finland-temperature-2013.nc
mescan-surfex-finland-temperature-2014.nc
mescan_surfex_nordics_2t.sql
mescan_surfex_nordics_consecutive_frost_days_index_per_time_period.sql
mescan_surfex_nordics_consecutive_summer_days_index_per_time_period.sql
mescan-surfex-no

In [9]:
files = [
    "mescan-surfex-nordics-eca_cfd.nc",
    "mescan-surfex-nordics-eca_csu.nc",
    "mescan-surfex-nordics-eca_fd.nc",
    "mescan-surfex-nordics-eca_hd.nc",
    "mescan-surfex-nordics-eca_id.nc",
    "mescan-surfex-nordics-eca_su.nc",
    "mescan-surfex-nordics-eca_tr.nc",
    "mescan-surfex-nordics-t2m_ymonstd.nc"
]

Build list of variables for each file

In [10]:
variables = {}
for f in files:
    ds = xr.open_dataset(path + f)
    for v in ds.variables:
        if f not in variables:
            variables[f] = []
        if v not in ["time", "lon", "lat", "inate.", "oordinate.", "height"]:
            variables[f].append(v)

print(variables)

{'mescan-surfex-nordics-eca_cfd.nc': ['consecutive_frost_days_index_per_time_period', 'number_of_cfd_periods_with_more_than_5days_per_time_period'], 'mescan-surfex-nordics-eca_csu.nc': ['consecutive_summer_days_index_per_time_period', 'number_of_csu_periods_with_more_than_5days_per_time_period'], 'mescan-surfex-nordics-eca_fd.nc': ['frost_days_index_per_time_period'], 'mescan-surfex-nordics-eca_hd.nc': ['heating_degree_days_per_time_period'], 'mescan-surfex-nordics-eca_id.nc': ['ice_days_index_per_time_period'], 'mescan-surfex-nordics-eca_su.nc': ['summer_days_index_per_time_period'], 'mescan-surfex-nordics-eca_tr.nc': ['tropical_nights_index_per_time_period'], 'mescan-surfex-nordics-t2m_ymonstd.nc': ['2t']}


Convert each variable to a single GeoTiff file

In [11]:
for f in files:
    print(f)
    for v in variables[f]:
        g = "%s.tiff" % v
        print(g) 
        call(["gdalwarp", "-s_srs", "EPSG:4326", "-t_srs", "EPSG:3035", "NETCDF:%s/%s:%s" % (path, f, v), 
              "-of", "GTiff", "%s/%s" % (path, g)])

mescan-surfex-nordics-eca_cfd.nc
consecutive_frost_days_index_per_time_period.tiff
number_of_cfd_periods_with_more_than_5days_per_time_period.tiff
mescan-surfex-nordics-eca_csu.nc
consecutive_summer_days_index_per_time_period.tiff
number_of_csu_periods_with_more_than_5days_per_time_period.tiff
mescan-surfex-nordics-eca_fd.nc
frost_days_index_per_time_period.tiff
mescan-surfex-nordics-eca_hd.nc
heating_degree_days_per_time_period.tiff
mescan-surfex-nordics-eca_id.nc
ice_days_index_per_time_period.tiff
mescan-surfex-nordics-eca_su.nc
summer_days_index_per_time_period.tiff
mescan-surfex-nordics-eca_tr.nc
tropical_nights_index_per_time_period.tiff
mescan-surfex-nordics-t2m_ymonstd.nc
2t.tiff


In [3]:
password = getpass()

········


In [12]:
conn = psycopg2.connect("dbname='ccgeodb' user='linda_ml' host='10.0.18.10' password='%s'" % (password))

Generate SQL script to load each GeoTiff, create tables in the database and store the SQL files

In [13]:
cursor = conn.cursor()
for f in files:
    for v in variables[f]:
        g = "%s.tiff" % v
        table = "mescan_surfex_nordics_%s" % g[0:-5]
        print(g)
        print(table)
        output = check_output(["raster2pgsql", "-I", "%s%s" % (path, g), "climate.%s" % table]).decode(sys.stdout.encoding)
        statements = output.split('\n')
    
        # skip BEGIN and COMMIT statements
        for i in range(1, 4):
            if i == 1:
                print(statements[i])
            cursor.execute(statements[i])

        conn.commit()

        
        with open('%s%s.sql' % (path, table), 'w') as sql:
            sql.write(output)

consecutive_frost_days_index_per_time_period.tiff
mescan_surfex_nordics_consecutive_frost_days_index_per_time_period
CREATE TABLE "climate"."mescan_surfex_nordics_consecutive_frost_days_index_per_time_period" ("rid" serial PRIMARY KEY,"rast" raster);
number_of_cfd_periods_with_more_than_5days_per_time_period.tiff
mescan_surfex_nordics_number_of_cfd_periods_with_more_than_5days_per_time_period
CREATE TABLE "climate"."mescan_surfex_nordics_number_of_cfd_periods_with_more_than_5days_per_time_period" ("rid" serial PRIMARY KEY,"rast" raster);
consecutive_summer_days_index_per_time_period.tiff
mescan_surfex_nordics_consecutive_summer_days_index_per_time_period
CREATE TABLE "climate"."mescan_surfex_nordics_consecutive_summer_days_index_per_time_period" ("rid" serial PRIMARY KEY,"rast" raster);
number_of_csu_periods_with_more_than_5days_per_time_period.tiff
mescan_surfex_nordics_number_of_csu_periods_with_more_than_5days_per_time_period
CREATE TABLE "climate"."mescan_surfex_nordics_number_of_c